<a href="https://colab.research.google.com/github/edumabagraham/aai/blob/main/MIIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! [ -e /content ] && pip install -Uqq fastbook
!pip install -Uq fastai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
from fastbook import *
setup_book()
from fastai.vision import *
import csv
from pathlib import Path
import os
import shutil
import pandas as pd


Mounted at /content/gdrive


In [3]:
# create test directory
dest = Path.cwd() / "test"
dest.mkdir(exist_ok=True)

# create pothole_or_not directory with sub-directories,
classes = "road", "pothole"
path = Path('pothole_or_not')

if not path.exists():
  for o in classes:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)

**Exploratory data analysis**

In [4]:
train_ids__labels_path = Path.cwd() /'gdrive/MyDrive/Colab Notebooks/MIIA/train_ids_labels.csv'
test_ids__only_path = Path.cwd() / 'gdrive/MyDrive/Colab Notebooks/MIIA/test_ids_only.csv'

In [5]:
train_ids_labels = pd.read_csv(train_ids__labels_path, low_memory=False)
test_ids_only = pd.read_csv(test_ids__only_path, low_memory=False)

In [6]:
# isinstance(train_ids_labels, pd.DataFrame)
train_ids_labels.iloc[975]

Image_ID    TReXBIUJikjnkob
Label                     1
Name: 975, dtype: object

In [7]:
print(train_ids_labels.shape,test_ids_only.shape)

(4026, 2) (1650, 1)


In [8]:
test_ids = pd.read_csv(test_ids__only_path).iloc[:, 0].tolist()

**Splitting data**

In [9]:
# Importing the csv into a dictionary
with open(f'{train_ids__labels_path}', mode='r', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)
    gt = {rows[0]:rows[1] for rows in reader}
print(len(gt))

4026


In [10]:
# setting path and list variables
path = Path.cwd() /'gdrive/MyDrive/Colab Notebooks/MIIA/all_data'

# Specify the path of the destination directory you want to copy to
test = '/content/test'
road = '/content/pothole_or_not/road'
pothole = '/content/pothole_or_not/pothole'

# loading dataset
for root, dirs, files in os.walk(path):
    for file in files:
          path = root+'//'+file
          if os.path.exists(path):
            file_name = file.split('.')[0]
            file_name = file_name.split(' ')[0]
            if file_name in test_ids:
              shutil.copy(path, test)
            else:
              if gt[file_name] == "1":
                shutil.copy(path, pothole)
              else:
                shutil.copy(path, road)
          else:
            print("Let's go again")

**Training model**

In [12]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)




TypeError: 'NoneType' object is not iterable

**Training**

In [ ]:
learn = vision_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(3)

**Predictions in csv**

In [ ]:
TEST_DATASET_PATH = Path('test/')

image_label_dict = {"Image_ID":"label"}
images = []
labels = []
for root, dirs, files in os.walk(TEST_DATASET_PATH):
  for file in files:
   label,_,probs = learn.predict(PILImage.create(f'{TEST_DATASET_PATH}/{file}'))
   file_name = file.split('.')[0]
   file_name = file_name.split(' ')[0]
   label = '1' if label == 'pothole' else '0'
   images.append(file_name)
   labels.append(label)


In [ ]:
df = pd.DataFrame(
   { 'Image_ID': images,
    'Label':labels
   }
)

In [ ]:
df.to_csv("Submission2.csv", index=False)